# 问题1：烟幕干扰弹对M1的有效遮蔽时长计算

## 问题描述
- 无人机：FY1
- 速度：120m/s（朝假目标方向）
- 投放时间：受领任务1.5秒后
- 起爆时间：投放后3.6秒
- 要求：计算对M1的有效遮蔽时长

基于03-01-A1-P1-单弹固定参数分析.md的建模思路

In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import json
from datetime import datetime
import os
from typing import Tuple, List, Union, Optional

# 确保输出目录存在
output_dir = "../ImageOutput/01"
os.makedirs(output_dir, exist_ok=True)

print("🚀 问题1：单弹固定参数分析")
print("=" * 50)

🚀 问题1：单弹固定参数分析


## 1. 物理参数定义

In [2]:
print("=== 第一步：定义基本参数 ===")

# 物理常量
g = 9.8  # 重力加速度 m/s²
smoke_sink_speed = 3.0  # 烟幕云团下沉速度 m/s
effective_radius = 10.0  # 有效遮蔽半径 m
effective_duration = 20.0  # 有效遮蔽持续时间 s

# 导弹参数
missile_speed = 300.0  # 导弹速度 m/s
M1_initial = np.array([20000.0, 0.0, 2000.0])  # 导弹M1初始位置

# 无人机参数
FY1_initial = np.array([17800.0, 0.0, 1800.0])  # 无人机FY1初始位置
drone_speed = 120.0  # 无人机速度 m/s

# 目标位置
fake_target = np.array([0.0, 0.0, 0.0])  # 假目标位置
real_target = np.array([0.0, 200.0, 0.0])  # 真目标位置

# 时间参数
t_deploy = 1.5  # 投放时间 s
t_explode_delay = 3.6  # 起爆延迟 s
t_explode = t_deploy + t_explode_delay  # 起爆时间 s

print(f"📊 物理参数初始化完成")
print(f"   导弹M1初始位置: {M1_initial}")
print(f"   无人机FY1初始位置: {FY1_initial}")
print(f"   真目标位置: {real_target}")
print(f"   投放时刻: {t_deploy}s, 起爆时刻: {t_explode}s")

=== 第一步：定义基本参数 ===
📊 物理参数初始化完成
   导弹M1初始位置: [20000.     0.  2000.]
   无人机FY1初始位置: [17800.     0.  1800.]
   真目标位置: [  0. 200.   0.]
   投放时刻: 1.5s, 起爆时刻: 5.1s


## 2. 运动学方程计算

In [3]:
print("\n=== 第二步：运动学方程计算 ===")

def calculate_missile_velocity(initial_pos: np.ndarray, target_pos: np.ndarray, speed: float) -> np.ndarray:
    """计算导弹速度向量"""
    direction = target_pos - initial_pos
    unit_direction = direction / np.linalg.norm(direction)
    return speed * unit_direction

def missile_position(t: float, initial_pos: np.ndarray, velocity: np.ndarray) -> np.ndarray:
    """计算导弹在时刻t的位置"""
    return initial_pos + velocity * t

def calculate_drone_velocity_horizontal(initial_pos: np.ndarray, target_pos: np.ndarray, speed: float) -> np.ndarray:
    """计算无人机水平方向速度向量（等高度飞行）"""
    # 只考虑xy平面的方向
    direction_2d = target_pos[:2] - initial_pos[:2]
    unit_direction_2d = direction_2d / np.linalg.norm(direction_2d)
    velocity_3d = np.array([unit_direction_2d[0], unit_direction_2d[1], 0.0]) * speed
    return velocity_3d

def drone_position(t: float, initial_pos: np.ndarray, velocity: np.ndarray) -> np.ndarray:
    """计算无人机在时刻t的位置"""
    return initial_pos + velocity * t

def smoke_bomb_position(t: float, deploy_time: float, deploy_pos: np.ndarray, 
                       initial_velocity: np.ndarray) -> np.ndarray:
    """计算烟幕弹在时刻t的位置（考虑重力）"""
    if t < deploy_time:
        return deploy_pos  # 投放前位置不变
    
    dt = t - deploy_time
    # 水平方向保持初始速度，竖直方向受重力影响
    horizontal_displacement = initial_velocity[:2] * dt
    vertical_displacement = initial_velocity[2] * dt - 0.5 * g * dt**2
    
    position = deploy_pos.copy()
    position[:2] += horizontal_displacement
    position[2] += vertical_displacement
    
    return position

def smoke_cloud_position(t: float, explode_time: float, explode_pos: np.ndarray) -> Optional[np.ndarray]:
    """计算烟幕云团在时刻t的位置"""
    if t < explode_time:
        return None  # 未起爆
    
    dt = t - explode_time
    if dt > effective_duration:
        return None  # 超过有效时间
    
    # 云团以3 m/s速度下沉
    position = explode_pos.copy()
    position[2] -= smoke_sink_speed * dt
    
    return position

# 计算轨迹参数
missile_velocity = calculate_missile_velocity(M1_initial, fake_target, missile_speed)
drone_velocity = calculate_drone_velocity_horizontal(FY1_initial, fake_target, drone_speed)
deploy_position = drone_position(t_deploy, FY1_initial, drone_velocity)
explode_position = smoke_bomb_position(t_explode, t_deploy, deploy_position, drone_velocity)

print(f"🎯 轨迹计算结果:")
print(f"   导弹速度向量: {missile_velocity}")
print(f"   无人机速度向量: {drone_velocity}")
print(f"   投放点: {deploy_position}")
print(f"   起爆位置: {explode_position}")


=== 第二步：运动学方程计算 ===
🎯 轨迹计算结果:
   导弹速度向量: [-298.51115706    0.          -29.85111571]
   无人机速度向量: [-120.    0.    0.]
   投放点: [17620.     0.  1800.]
   起爆位置: [17188.        0.     1736.496]


## 3. 遮蔽判定函数

In [4]:
print("\n=== 第三步：遮蔽判定函数 ===")

def point_to_line_segment_distance(point: np.ndarray, line_start: np.ndarray, 
                                 line_end: np.ndarray) -> Tuple[float, float]:
    """
    计算点到线段的最短距离
    返回: (距离, 投影参数u)
    """
    # 向量计算
    AB = line_end - line_start
    AP = point - line_start
    
    # 投影参数
    AB_squared = np.dot(AB, AB)
    if AB_squared == 0:
        return float(np.linalg.norm(AP)), 0.0
    
    u = float(np.dot(AP, AB) / AB_squared)
    
    # 计算距离
    if u < 0:
        # 最近点在线段起点之外
        distance = float(np.linalg.norm(AP))
    elif u > 1:
        # 最近点在线段终点之外
        BP = point - line_end
        distance = float(np.linalg.norm(BP))
    else:
        # 最近点在线段上
        cross_product = np.cross(AP, AB)
        if AB.ndim == 1 and len(AB) == 3:
            distance = float(np.linalg.norm(cross_product) / np.linalg.norm(AB))
        else:
            distance = float(abs(cross_product) / np.linalg.norm(AB))
    
    return distance, u

def is_shielded(t: float, explode_time: float, explode_pos: np.ndarray,
               missile_initial: np.ndarray, missile_vel: np.ndarray,
               target_pos: np.ndarray, radius: float) -> Tuple[bool, dict]:
    """
    判断在时刻t是否被遮蔽
    返回: (是否遮蔽, 详细信息字典)
    """
    # 检查云团是否有效
    if t < explode_time or t > explode_time + effective_duration:
        return False, {"reason": "云团无效"}
    
    # 计算各位置
    missile_pos = missile_position(t, missile_initial, missile_vel)
    cloud_pos = smoke_cloud_position(t, explode_time, explode_pos)
    
    if cloud_pos is None:
        return False, {"reason": "云团位置无效"}
    
    # 计算点到线段距离
    distance, u = point_to_line_segment_distance(cloud_pos, missile_pos, target_pos)
    
    # 判断遮蔽条件
    is_blocked = distance <= radius and 0 <= u <= 1
    
    info = {
        "time": t,
        "missile_pos": missile_pos,
        "cloud_pos": cloud_pos,
        "target_pos": target_pos,
        "distance": distance,
        "projection_u": u,
        "is_blocked": is_blocked
    }
    
    return is_blocked, info

print("✅ 遮蔽判定函数定义完成")


=== 第三步：遮蔽判定函数 ===
✅ 遮蔽判定函数定义完成


## 4. 数值求解遮蔽时长

In [5]:
print("\n=== 第四步：数值求解遮蔽时长 ===")

def calculate_shielding_duration(explode_time: float, explode_pos: np.ndarray,
                               missile_initial: np.ndarray, missile_vel: np.ndarray,
                               target_pos: np.ndarray, radius: float,
                               dt: float = 0.01) -> Tuple[float, List[dict]]:
    """
    数值计算有效遮蔽时长
    返回: (总遮蔽时长, 详细记录列表)
    """
    # 时间范围
    t_start = explode_time
    t_end = explode_time + effective_duration
    
    # 时间采样
    time_points = np.arange(t_start, t_end + dt, dt)
    
    shielded_count = 0
    detailed_records = []
    all_records = []  # 记录所有时间点的数据
    
    for t in time_points:
        blocked, info = is_shielded(float(t), explode_time, explode_pos,
                                  missile_initial, missile_vel, target_pos, radius)
        
        all_records.append({
            'time': float(t),
            'distance': info.get('distance', float('inf')),
            'projection_u': info.get('projection_u', 0.0),
            'is_blocked': blocked
        })
        
        if blocked:
            shielded_count += 1
            detailed_records.append(info)
    
    total_duration = shielded_count * dt
    
    # 找到遮蔽区间
    shielded_intervals = []
    in_interval = False
    interval_start = None
    
    for record in all_records:
        t = record['time']
        shielded = record['is_blocked']
        
        if shielded and not in_interval:
            # 开始遮蔽
            interval_start = t
            in_interval = True
        elif not shielded and in_interval:
            # 结束遮蔽
            shielded_intervals.append((interval_start, t - dt))
            in_interval = False
    
    # 处理最后一个区间
    if in_interval:
        shielded_intervals.append((interval_start, time_points[-1]))
    
    return total_duration, detailed_records, all_records, shielded_intervals

# 求解遮蔽时长
shielding_duration, records, all_data, intervals = calculate_shielding_duration(
    t_explode, explode_position, M1_initial, missile_velocity, 
    real_target, effective_radius
)

print(f"🎯 遮蔽分析结果:")
print(f"   总遮蔽时长: {shielding_duration:.6f}s")
print(f"   遮蔽区间数量: {len(intervals)}")

for i, (start, end) in enumerate(intervals):
    print(f"   区间{i+1}: {start:.3f}s - {end:.3f}s (时长: {end-start:.3f}s)")


=== 第四步：数值求解遮蔽时长 ===
🎯 遮蔽分析结果:
   总遮蔽时长: 1.380000s
   遮蔽区间数量: 1
   区间1: 8.040s - 9.410s (时长: 1.370s)


## 5. 3D轨迹可视化

In [6]:
def create_3d_trajectory_plot():
    """创建3D轨迹可视化"""
    
    fig = go.Figure()
    
    # 时间范围
    t_max = 30.0
    t_trajectory = np.linspace(0, t_max, 200)
    
    # 导弹轨迹
    missile_trajectory = np.array([missile_position(t, M1_initial, missile_velocity) for t in t_trajectory])
    fig.add_trace(go.Scatter3d(
        x=missile_trajectory[:, 0],
        y=missile_trajectory[:, 1],
        z=missile_trajectory[:, 2],
        mode='lines',
        line=dict(color='red', width=6),
        name='导弹M1轨迹',
        hovertemplate='<b>导弹M1</b><br>坐标: (%{x:.0f}, %{y:.0f}, %{z:.0f})<extra></extra>'
    ))
    
    # 无人机轨迹（到投放点）
    t_drone = np.linspace(0, t_deploy, 50)
    drone_trajectory = np.array([drone_position(t, FY1_initial, drone_velocity) for t in t_drone])
    fig.add_trace(go.Scatter3d(
        x=drone_trajectory[:, 0],
        y=drone_trajectory[:, 1],
        z=drone_trajectory[:, 2],
        mode='lines',
        line=dict(color='blue', width=6),
        name='无人机FY1轨迹',
        hovertemplate='<b>无人机FY1</b><br>坐标: (%{x:.0f}, %{y:.0f}, %{z:.0f})<extra></extra>'
    ))
    
    # 烟幕弹轨迹（投放到起爆）
    t_smoke = np.linspace(t_deploy, t_explode, 50)
    smoke_trajectory = np.array([smoke_bomb_position(t, t_deploy, deploy_position, drone_velocity) for t in t_smoke])
    
    fig.add_trace(go.Scatter3d(
        x=smoke_trajectory[:, 0],
        y=smoke_trajectory[:, 1],
        z=smoke_trajectory[:, 2],
        mode='lines',
        line=dict(color='orange', width=4, dash='dash'),
        name='烟幕弹轨迹',
        hovertemplate='<b>烟幕弹</b><br>坐标: (%{x:.0f}, %{y:.0f}, %{z:.0f})<extra></extra>'
    ))
    
    # 云团轨迹（起爆后下沉）
    t_cloud = np.linspace(t_explode, t_explode + effective_duration, 100)
    cloud_trajectory = []
    for t in t_cloud:
        pos = smoke_cloud_position(t, t_explode, explode_position)
        if pos is not None:
            cloud_trajectory.append(pos)
    cloud_trajectory = np.array(cloud_trajectory)
    
    fig.add_trace(go.Scatter3d(
        x=cloud_trajectory[:, 0],
        y=cloud_trajectory[:, 1],
        z=cloud_trajectory[:, 2],
        mode='lines',
        line=dict(color='gray', width=8),
        name='云团中心轨迹',
        hovertemplate='<b>云团中心</b><br>坐标: (%{x:.0f}, %{y:.0f}, %{z:.0f})<extra></extra>'
    ))
    
    # 关键点标记
    # 初始位置
    fig.add_trace(go.Scatter3d(
        x=[M1_initial[0]], y=[M1_initial[1]], z=[M1_initial[2]],
        mode='markers',
        marker=dict(size=12, color='red', symbol='diamond'),
        name='M1初始位置',
        hovertemplate='<b>M1初始位置</b><br>坐标: (%{x:.0f}, %{y:.0f}, %{z:.0f})<extra></extra>'
    ))
    
    fig.add_trace(go.Scatter3d(
        x=[FY1_initial[0]], y=[FY1_initial[1]], z=[FY1_initial[2]],
        mode='markers',
        marker=dict(size=10, color='blue', symbol='circle'),
        name='FY1初始位置',
        hovertemplate='<b>FY1初始位置</b><br>坐标: (%{x:.0f}, %{y:.0f}, %{z:.0f})<extra></extra>'
    ))
    
    # 投放点
    fig.add_trace(go.Scatter3d(
        x=[deploy_position[0]], y=[deploy_position[1]], z=[deploy_position[2]],
        mode='markers',
        marker=dict(size=8, color='orange', symbol='square'),
        name='投放点',
        hovertemplate='<b>投放点</b><br>坐标: (%{x:.0f}, %{y:.0f}, %{z:.0f})<extra></extra>'
    ))
    
    # 起爆点
    fig.add_trace(go.Scatter3d(
        x=[explode_position[0]], y=[explode_position[1]], z=[explode_position[2]],
        mode='markers',
        marker=dict(size=10, color='purple', symbol='diamond'),
        name='起爆点',
        hovertemplate='<b>起爆点</b><br>坐标: (%{x:.0f}, %{y:.0f}, %{z:.0f})<extra></extra>'
    ))
    
    # 真目标
    fig.add_trace(go.Scatter3d(
        x=[real_target[0]], y=[real_target[1]], z=[real_target[2]],
        mode='markers',
        marker=dict(size=15, color='green', symbol='cross'),
        name='真目标',
        hovertemplate='<b>真目标</b><br>坐标: (%{x:.0f}, %{y:.0f}, %{z:.0f})<extra></extra>'
    ))
    
    # 假目标（原点）
    fig.add_trace(go.Scatter3d(
        x=[0], y=[0], z=[0],
        mode='markers',
        marker=dict(size=12, color='black', symbol='x'),
        name='假目标',
        hovertemplate='<b>假目标</b><br>坐标: (0, 0, 0)<extra></extra>'
    ))
    
    # 设置布局
    fig.update_layout(
        title=dict(
            text='🚀 问题1：3D轨迹可视化<br><sub>单弹固定参数分析</sub>',
            x=0.5,
            font=dict(size=20, color='darkblue')
        ),
        scene=dict(
            xaxis_title='X坐标 (m)',
            yaxis_title='Y坐标 (m)',
            zaxis_title='Z坐标 (m)',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.2)
            ),
            aspectmode='manual',
            aspectratio=dict(x=2, y=1, z=0.5)
        ),
        width=1200,
        height=800,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )
    
    return fig

# 创建3D轨迹图
fig_3d = create_3d_trajectory_plot()
fig_3d.show()

# 保存图像
fig_3d.write_html(f"{output_dir}/01_3d_trajectory.html")
fig_3d.write_image(f"{output_dir}/01_3d_trajectory.png", width=1200, height=800, scale=2)
print(f"💾 3D轨迹图已保存到 {output_dir}/01_3d_trajectory.html")

💾 3D轨迹图已保存到 ../ImageOutput/01/01_3d_trajectory.html


## 6. 遮蔽距离时间序列分析

In [7]:
def create_shielding_analysis_plot():
    """创建遮蔽分析图表"""
    
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('云团与导弹-目标视线的距离随时间变化', '遮蔽状态时间序列'),
        vertical_spacing=0.12,
        specs=[[{"secondary_y": False}], [{"secondary_y": False}]]
    )
    
    # 准备数据
    times = [record['time'] for record in all_data]
    distances = [record['distance'] for record in all_data]
    shielded_flags = [record['is_blocked'] for record in all_data]
    
    # 第一个子图：距离曲线
    fig.add_trace(
        go.Scatter(
            x=times,
            y=distances,
            mode='lines',
            line=dict(color='blue', width=2),
            name='距离d(t)',
            hovertemplate='时间: %{x:.2f}s<br>距离: %{y:.2f}m<extra></extra>'
        ),
        row=1, col=1
    )
    
    # 遮蔽阈值线
    fig.add_trace(
        go.Scatter(
            x=[times[0], times[-1]],
            y=[effective_radius, effective_radius],
            mode='lines',
            line=dict(color='red', width=2, dash='dash'),
            name=f'遮蔽阈值 R={effective_radius}m',
            hovertemplate='遮蔽阈值: %{y}m<extra></extra>'
        ),
        row=1, col=1
    )
    
    # 遮蔽区域填充
    shielded_distances = [d if s else np.nan for d, s in zip(distances, shielded_flags)]
    fig.add_trace(
        go.Scatter(
            x=times,
            y=shielded_distances,
            mode='lines',
            line=dict(color='green', width=3),
            name='有效遮蔽区间',
            fill='tonexty',
            fillcolor='rgba(0,255,0,0.2)',
            hovertemplate='时间: %{x:.2f}s<br>遮蔽距离: %{y:.2f}m<extra></extra>'
        ),
        row=1, col=1
    )
    
    # 第二个子图：遮蔽状态
    fig.add_trace(
        go.Scatter(
            x=times,
            y=[int(flag) for flag in shielded_flags],
            mode='lines',
            line=dict(color='green', width=3),
            name='遮蔽状态',
            fill='tozeroy',
            fillcolor='rgba(0,255,0,0.3)',
            hovertemplate='时间: %{x:.2f}s<br>遮蔽状态: %{y}<extra></extra>'
        ),
        row=2, col=1
    )
    
    # 标记遮蔽区间
    for i, (start, end) in enumerate(intervals):
        fig.add_vrect(
            x0=start, x1=end,
            fillcolor="green", opacity=0.2,
            layer="below", line_width=0,
            row=1, col=1
        )
        
        # 添加区间标注
        mid_time = (start + end) / 2
        fig.add_annotation(
            x=mid_time,
            y=effective_radius * 0.5,
            text=f"区间{i+1}<br>{end-start:.3f}s",
            showarrow=True,
            arrowhead=2,
            arrowcolor="green",
            font=dict(color="green", size=10),
            row=1, col=1
        )
    
    # 更新布局
    fig.update_layout(
        title=dict(
            text=f'📊 遮蔽效果分析<br><sub>总遮蔽时长: {shielding_duration:.6f}s</sub>',
            x=0.5,
            font=dict(size=18, color='darkblue')
        ),
        height=800,
        showlegend=True,
        hovermode='x unified'
    )
    
    # 更新坐标轴
    fig.update_xaxes(title_text="时间 (s)", row=1, col=1)
    fig.update_yaxes(title_text="距离 (m)", row=1, col=1)
    fig.update_xaxes(title_text="时间 (s)", row=2, col=1)
    fig.update_yaxes(title_text="遮蔽状态", row=2, col=1, tickvals=[0, 1], ticktext=['未遮蔽', '遮蔽'])
    
    return fig

# 创建遮蔽分析图
fig_analysis = create_shielding_analysis_plot()
fig_analysis.show()

# 保存图像
fig_analysis.write_html(f"{output_dir}/02_shielding_analysis.html")
fig_analysis.write_image(f"{output_dir}/02_shielding_analysis.png", width=1200, height=800, scale=2)
print(f"💾 遮蔽分析图已保存到 {output_dir}/02_shielding_analysis.html")

WARNING	Thread(Thread-6 (run)) Task(Task-41) choreographer.browser_async:browser_async.py:_close()- Resorting to unclean kill browser.


💾 遮蔽分析图已保存到 ../ImageOutput/01/02_shielding_analysis.html


## 7. 参数敏感性分析

In [ ]:
def create_parameter_sensitivity_plot():
    """创建参数敏感性分析图"""
    
    # 分析不同起爆延时的影响
    delay_range = np.arange(2.0, 6.0, 0.2)
    durations = []
    
    for delay in delay_range:
        test_explode_time = t_deploy + delay
        test_explode_pos = smoke_bomb_position(test_explode_time, t_deploy, deploy_position, drone_velocity)
        
        duration, _, _, _ = calculate_shielding_duration(
            test_explode_time, test_explode_pos, M1_initial, missile_velocity, 
            real_target, effective_radius
        )
        durations.append(duration)
    
    # 创建敏感性分析图
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=delay_range,
        y=durations,
        mode='lines+markers',
        line=dict(color='blue', width=3),
        marker=dict(size=8, color='red'),
        name='遮蔽时长',
        hovertemplate='起爆延时: %{x:.1f}s<br>遮蔽时长: %{y:.3f}s<extra></extra>'
    ))
    
    # 标记当前参数
    fig.add_trace(go.Scatter(
        x=[t_explode_delay],
        y=[shielding_duration],
        mode='markers',
        marker=dict(size=15, color='green', symbol='star'),
        name=f'当前参数 ({t_explode_delay}s)',
        hovertemplate=f'当前起爆延时: {t_explode_delay}s<br>遮蔽时长: {shielding_duration:.6f}s<extra></extra>'
    ))
    
    fig.update_layout(
        title=dict(
            text='📈 参数敏感性分析<br><sub>起爆延时对遮蔽时长的影响</sub>',
            x=0.5,
            font=dict(size=18, color='darkblue')
        ),
        xaxis_title='起爆延时 (s)',
        yaxis_title='遮蔽时长 (s)',
        width=1000,
        height=600,
        showlegend=True
    )
    
    return fig

# 创建参数敏感性分析图
fig_sensitivity = create_parameter_sensitivity_plot()
fig_sensitivity.show()

# 保存图像
fig_sensitivity.write_html(f"{output_dir}/03_parameter_sensitivity.html")
fig_sensitivity.write_image(f"{output_dir}/03_parameter_sensitivity.png", width=1000, height=600, scale=2)
print(f"💾 参数敏感性分析图已保存到 {output_dir}/03_parameter_sensitivity.html")

## 8. 结果汇总与保存

In [ ]:
def create_results_summary():
    """创建结果汇总"""
    
    summary = {
        "问题": "问题1：单弹固定参数分析",
        "计算时间": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "物理参数": {
            "导弹M1初始位置": M1_initial.tolist(),
            "导弹速度": f"{missile_speed} m/s",
            "无人机FY1初始位置": FY1_initial.tolist(),
            "无人机速度": f"{drone_speed} m/s",
            "真目标位置": real_target.tolist(),
            "投放时间": f"{t_deploy} s",
            "起爆延时": f"{t_explode_delay} s",
            "起爆时刻": f"{t_explode} s",
            "烟幕有效半径": f"{effective_radius} m",
            "烟幕有效时间": f"{effective_duration} s",
            "云团下沉速度": f"{smoke_sink_speed} m/s"
        },
        "计算结果": {
            "导弹速度向量": missile_velocity.tolist(),
            "无人机速度向量": drone_velocity.tolist(),
            "投放点坐标": deploy_position.tolist(),
            "起爆点坐标": explode_position.tolist(),
            "总遮蔽时长": f"{shielding_duration:.6f} s",
            "遮蔽区间数量": len(intervals),
            "遮蔽区间详情": [
                {
                    "区间": i+1,
                    "开始时间": f"{start:.6f} s",
                    "结束时间": f"{end:.6f} s",
                    "持续时间": f"{end-start:.6f} s"
                }
                for i, (start, end) in enumerate(intervals)
            ],
            "数值计算参数": {
                "时间步长": "0.01 s",
                "分析时间范围": f"{t_explode:.1f}s - {t_explode + effective_duration:.1f}s"
            }
        }
    }
    
    return summary

# 创建结果汇总
summary = create_results_summary()

# 保存结果到JSON文件
with open(f"{output_dir}/04_results_summary.json", 'w', encoding='utf-8') as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

# 创建结果表格
results_df = pd.DataFrame([
    ["导弹M1初始位置", f"({M1_initial[0]}, {M1_initial[1]}, {M1_initial[2]})"],
    ["无人机FY1初始位置", f"({FY1_initial[0]}, {FY1_initial[1]}, {FY1_initial[2]})"],
    ["真目标位置", f"({real_target[0]}, {real_target[1]}, {real_target[2]})"],
    ["投放点坐标", f"({deploy_position[0]:.2f}, {deploy_position[1]:.2f}, {deploy_position[2]:.2f})"],
    ["起爆点坐标", f"({explode_position[0]:.2f}, {explode_position[1]:.2f}, {explode_position[2]:.2f})"],
    ["投放时刻", f"{t_deploy} s"],
    ["起爆时刻", f"{t_explode} s"],
    ["总遮蔽时长", f"{shielding_duration:.6f} s"],
    ["遮蔽区间数量", f"{len(intervals)}个"]
], columns=["参数", "数值"])

# 保存结果表格
results_df.to_csv(f"{output_dir}/05_results_table.csv", index=False, encoding='utf-8-sig')
results_df.to_excel(f"{output_dir}/05_results_table.xlsx", index=False)

print("📋 问题1计算结果汇总:")
print("=" * 50)
print(results_df.to_string(index=False))
print("=" * 50)
print(f"🎯 **最终答案：对M1的有效遮蔽时长为 {shielding_duration:.6f} 秒**")
print("=" * 50)

# 保存详细数据
detailed_data = pd.DataFrame(all_data)
detailed_data.to_csv(f"{output_dir}/06_detailed_data.csv", index=False)

print(f"💾 所有结果已保存到 {output_dir}/ 目录")
print(f"   - 3D轨迹图: 01_3d_trajectory.html")
print(f"   - 遮蔽分析图: 02_shielding_analysis.html") 
print(f"   - 参数敏感性分析: 03_parameter_sensitivity.html")
print(f"   - 结果汇总: 04_results_summary.json")
print(f"   - 结果表格: 05_results_table.xlsx")
print(f"   - 详细数据: 06_detailed_data.csv")

## 9. 最终结果展示

In [ ]:
# 创建最终结果展示图表
def create_final_summary_plot():
    """创建最终结果展示"""
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '关键时间节点',
            '关键位置坐标', 
            '遮蔽效果统计',
            '计算参数汇总'
        ),
        specs=[
            [{"type": "bar"}, {"type": "scatter3d"}],
            [{"type": "pie"}, {"type": "table"}]
        ]
    )
    
    # 关键时间节点
    time_labels = ['投放时间', '起爆时间', '遮蔽开始', '遮蔽结束', '云团消失']
    time_values = [
        t_deploy, 
        t_explode, 
        intervals[0][0] if intervals else 0,
        intervals[0][1] if intervals else 0,
        t_explode + effective_duration
    ]
    
    fig.add_trace(
        go.Bar(
            x=time_labels,
            y=time_values,
            marker_color=['blue', 'orange', 'green', 'red', 'gray'],
            name='时间节点'
        ),
        row=1, col=1
    )
    
    # 遮蔽效果统计
    total_time = effective_duration
    shielded_time = shielding_duration
    unshielded_time = total_time - shielded_time
    
    fig.add_trace(
        go.Pie(
            labels=['有效遮蔽', '未遮蔽'],
            values=[shielded_time, unshielded_time],
            marker_colors=['green', 'lightgray'],
            name='遮蔽效果'
        ),
        row=2, col=1
    )
    
    # 计算参数表格
    param_data = [
        ['参数', '数值'],
        ['遮蔽时长', f'{shielding_duration:.6f} s'],
        ['遮蔽效率', f'{shielded_time/total_time*100:.2f}%'],
        ['起爆高度', f'{explode_position[2]:.1f} m'],
        ['最小距离', f'{min([r["distance"] for r in records]):.3f} m' if records else 'N/A'],
        ['平均距离', f'{np.mean([r["distance"] for r in records]):.3f} m' if records else 'N/A']
    ]
    
    fig.add_trace(
        go.Table(
            header=dict(values=param_data[0], fill_color='lightblue'),
            cells=dict(values=list(zip(*param_data[1:])), fill_color='white')
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        title=dict(
            text=f'📊 问题1最终结果汇总<br><sub>有效遮蔽时长: {shielding_duration:.6f} 秒</sub>',
            x=0.5,
            font=dict(size=20, color='darkblue')
        ),
        height=800,
        showlegend=False
    )
    
    return fig

# 创建最终结果图
fig_final = create_final_summary_plot()
fig_final.show()

# 保存图像
fig_final.write_html(f"{output_dir}/07_final_summary.html")
fig_final.write_image(f"{output_dir}/07_final_summary.png", width=1200, height=800, scale=2)
print(f"💾 最终结果汇总图已保存到 {output_dir}/07_final_summary.html")

print("\n🎉 问题1分析完成！")
print(f"🎯 最终答案：烟幕干扰弹对M1的有效遮蔽时长为 {shielding_duration:.6f} 秒")